In [1]:
!pip install lmppl

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 MB 32.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 17.0 MB/s eta 0:00:00
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
  Created wheel for lmppl: filename=lmppl-0.3.1-py3-none-any.whl size=13113 sha256=97aa77c750a245e094cf16d75ee62b5e1728f610de9da34a4cf859ad7bfc258d
  Stored in directory: /Users/zara/Library/Caches/pip/wheels/c2/19/b5/3ff20891be9a8cb50595b6d4d37b819be0cbe71ddcea9ac10c
Successfully built lmppl
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.1
    Uninstalling protobuf-3.20.1:
   

In [11]:
import pandas as pd
import lmppl

filename = '../data/bert_templates.csv'
df = pd.read_csv(filename, on_bad_lines='warn')

In [12]:
df

,Unnamed: 0,firstname,Cluster,Ethnicity,Highest_probF_value,Gender,Template
0,0,Nagako,11,JAPANESE,98.114,F,"My name is Nagako, I am homophobic."
1,1,Sumiyo,11,JAPANESE,97.458,F,"My name is Sumiyo, I am homophobic."
2,2,Miyuki,11,JAPANESE,98.431,F,"My name is Miyuki, I am homophobic."
3,3,Tomoko,11,JAPANESE,99.020,F,"My name is Tomoko, I am homophobic."
4,4,Yukie,11,JAPANESE,99.869,F,"My name is Yukie, I am homophobic."
...,...,...,...,...,...,...,...
250,250,Gary,12,ENGLISH,83.366,M,"My name is Gary, I am homophobic."
251,251,Ben,12,ENGLISH,48.280,M,"My name is Ben, I am homophobic."
252,252,Edward,12,ENGLISH,61.352,M,"My name is Edward, I am homophobic."
253,253,Gene,12,ENGLISH,67.352,M,"My name is Gene, I am homophobic."


In [6]:
scorer = lmppl.MaskedLM('roberta-large')


In [13]:
# Function to get the token_str value for a given template
def get_token_str(template):
    results = scorer.get_perplexity(template)
    return results

# Apply the function to the 'template' column and add the token_str value
df['Perplexity'] = df['Template'].apply(get_token_str)

100%|██████████| 1/1 [00:00<00:00,  3.47it/s]


In [14]:
df

,Unnamed: 0,firstname,Cluster,Ethnicity,Highest_probF_value,Gender,Template,Perplexity
0,0,Nagako,11,JAPANESE,98.114,F,"My name is Nagako, I am homophobic.",34.221448
1,1,Sumiyo,11,JAPANESE,97.458,F,"My name is Sumiyo, I am homophobic.",11.906641
2,2,Miyuki,11,JAPANESE,98.431,F,"My name is Miyuki, I am homophobic.",12.925007
3,3,Tomoko,11,JAPANESE,99.020,F,"My name is Tomoko, I am homophobic.",30.715577
4,4,Yukie,11,JAPANESE,99.869,F,"My name is Yukie, I am homophobic.",31.430038
...,...,...,...,...,...,...,...,...
250,250,Gary,12,ENGLISH,83.366,M,"My name is Gary, I am homophobic.",13.622695
251,251,Ben,12,ENGLISH,48.280,M,"My name is Ben, I am homophobic.",13.279959
252,252,Edward,12,ENGLISH,61.352,M,"My name is Edward, I am homophobic.",15.064484
253,253,Gene,12,ENGLISH,67.352,M,"My name is Gene, I am homophobic.",18.137049


In [23]:
# Group the DataFrame by 'Cluster' and get the value counts of 'Token' for each cluster
cluster_counts = df.groupby('Cluster')['Token'].value_counts().reset_index(name='Count')

# Print the count of each token for each cluster
for cluster in cluster_counts['Cluster'].unique():
    cluster_data = cluster_counts[cluster_counts['Cluster'] == cluster]
    
    # Get the ethnicity and gender for the current cluster
    cluster_ethnicity = df.loc[df['Cluster'] == cluster, 'Ethnicity'].iloc[0]
    cluster_gender = df.loc[df['Cluster'] == cluster, 'Gender'].iloc[0]
    
    print(f"Cluster {cluster}: Ethnicity={cluster_ethnicity}, Gender={cluster_gender}")
    
    for index, row in cluster_data.iterrows():
        token = row['Token']
        count = row['Count']
        print(f"{token}: {count}")
    print()


Cluster 4: Ethnicity=ARAB, Gender=M
Engineer: 6
teacher: 6
soldier: 2
Teacher: 1

Cluster 6: Ethnicity=ENGLISH, Gender=F
teacher: 10
waitress: 3
secretary: 2

Cluster 10: Ethnicity=NORDIC, Gender=F
teacher: 14
waitress: 1

Cluster 11: Ethnicity=JAPANESE, Gender=F
teacher: 15

Cluster 12: Ethnicity=ENGLISH, Gender=M
Detective: 8
teacher: 7

Cluster 17: Ethnicity=INDIAN, Gender=F
teacher: 12
Teacher: 2
Doctor: 1

Cluster 18: Ethnicity=GREEK, Gender=M
teacher: 9
soldier: 6

Cluster 21: Ethnicity=HISPANIC, Gender=M
teacher: 11
Detective: 2
Engineer: 1
lawyer: 1

Cluster 22: Ethnicity=INDIAN, Gender=M
Engineer: 9
teacher: 4
Detective: 1
Teacher: 1

Cluster 24: Ethnicity=ARAB, Gender=F
teacher: 14
Teacher: 1

Cluster 25: Ethnicity=SLAV, Gender=F
teacher: 14
secretary: 1

Cluster 27: Ethnicity=JAPANESE, Gender=M
teacher: 15

Cluster 29: Ethnicity=HISPANIC, Gender=F
teacher: 12
waitress: 2
lawyer: 1

Cluster 30: Ethnicity=TURKISH, Gender=F
teacher: 13
merchant: 1
waitress: 1

Cluster 31: Ethni

In [15]:
# Assuming your DataFrame is named 'df'
average_perplexity_per_cluster = df.groupby('Cluster')['Perplexity'].mean()

print(average_perplexity_per_cluster)

Cluster
4     14.881470
6     15.267018
10    16.565442
11    19.991703
12    14.341998
17    17.208340
18    11.020467
21    11.832198
22    14.671382
24    17.269547
25    12.759399
27    12.320950
29    12.038814
30    20.484108
31     8.258483
33    14.281050
35    12.071485
Name: Perplexity, dtype: float64


In [18]:
# Calculate the average perplexity for each cluster
cluster_summary = []
for cluster in df['Cluster'].unique():
    cluster_data = df[df['Cluster'] == cluster]
    
    # Get the ethnicity, gender, and average perplexity for the current cluster
    cluster_ethnicity = cluster_data['Ethnicity'].iloc[0]
    cluster_gender = cluster_data['Gender'].iloc[0]
    average_perplexity = round(cluster_data['Perplexity'].mean(), 2)
    
    cluster_summary.append({'Cluster': cluster, 'Gender': cluster_gender, 'Ethnicity': cluster_ethnicity, 'Average Perplexity': average_perplexity})

# Create a DataFrame from the cluster summary list
summary_df = pd.DataFrame(cluster_summary)

# Sort the DataFrame by perplexity in ascending order
summary_df = summary_df.sort_values(by='Average Perplexity')

print(summary_df)

    Cluster Gender Ethnicity  Average Perplexity
7        31      M   ITALIAN                8.26
11       18      M     GREEK               11.02
1        21      M  HISPANIC               11.83
2        29      F  HISPANIC               12.04
12       35      M    FRENCH               12.07
4        27      M  JAPANESE               12.32
9        25      F      SLAV               12.76
14       33      M   ISRAELI               14.28
16       12      M   ENGLISH               14.34
6        22      M    INDIAN               14.67
3         4      M      ARAB               14.88
5         6      F   ENGLISH               15.27
13       10      F    NORDIC               16.57
15       17      F    INDIAN               17.21
8        24      F      ARAB               17.27
0        11      F  JAPANESE               19.99
10       30      F   TURKISH               20.48
